In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#from darksky import forecast
import datetime as dt
#from noaa_sdk import noaa
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
import pickle
from shapely.geometry import Point, Polygon

geo_data = pd.read_csv('data/distance_to_lake.csv')

scaler = RobustScaler()

geo_data['log_num'] = np.log(geo_data['num'])
feature_data = geo_data[['log_num','num','d_water','d_park',
       'd_lm','TL1','Tmin','Tmax','TH0','prec']].dropna()

feature_data['sqrt_dw']=np.sqrt(feature_data['d_water'])
feature_data['sqrt_dp']=np.sqrt(feature_data['d_park'])
X = feature_data[['TL1','Tmin','Tmax','d_lm','d_water','d_park','TH0','prec']]
X.columns
trainX = scaler.fit_transform(X)
trainy = feature_data.num
#las = ElasticNet(alpha = 1,l1_ratio=0.5).fit(trainX, trainy)
lin = LinearRegression().fit(trainX,trainy)
lin.score(trainX,trainy)

coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(lin.coef_))], axis = 1)
coefficients.columns
print(mean_absolute_error((trainy),lin.predict(trainX)))
#print(mean_absolute_error(trainy,lin.predict(trainX)))
#coefs = abs(coefficients.iloc[:,1])
print(coefficients)
modelfilename = 'data/lin_model.sav'
scalerfilename = 'data/lin_scaler.sav'
pickle.dump(lin, open(modelfilename, 'wb'))
pickle.dump(scaler, open(scalerfilename, 'wb'))

10.645863844562136
         0         0
0      TL1  1.220404
1     Tmin  2.266367
2     Tmax  0.466391
3     d_lm  3.085175
4  d_water -2.638007
5   d_park  3.125438
6      TH0  0.055581
7     prec  0.051480
